# Auth Code Flow using Certificate + PKCE
- In this notebook we will use Auth Code Flow, but this time with a certificate to authenticate the client, for setup see the `/Cert_Setup` directory.   
- We will also execute the Auth Code Flow paired with Proof Key for Code Exchange (PKCE), which includes the `code_challenge` and `code_challenge_method`.


In [2]:
import sys
sys.path.append('../')
import OAuth2_Flows
import pyperclip
import base64
import os
import re
import hashlib

## Required Variables Setup
Below we are setting up our variables.
- Note that the `redirect_uri` needs to match that on the App Registration

> Note that in the scope you could also request something like: `scope = 'openid email profile offline_access https://graph.microsoft.com/.default'`, and that would also return an id token along with a refresh token. This by the specification is no longer considered Auth Code flow, and it is using the Hybrid Flow leveraging the OpenID Connect (OIDC) spec, which allows for this type of behavior.

In [ ]:
tenant_id = ''
client_id = ''
redirect_uri = ''

scope = 'offline_access https://graph.microsoft.com/.default' #offline_access is required for refresh token
state = "A1B2C3D4E5F6"

---     
# PKCE code support


In [ ]:
#source: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/code-challenge/
def vGenerateCodeChallengePair() -> tuple:
  code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
  code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)

  code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
  code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
  code_challenge = code_challenge.replace('=', '')

  return (code_verifier, code_challenge)

code_verifier, code_challenge = vGenerateCodeChallengePair()
print(code_verifier)
print(code_challenge)
pyperclip.copy(code_challenge)

In [ ]:
complete_auth_url = OAuth2_Flows.auth_code_flow(tenant_id, client_id, redirect_uri, scope, state, code_challenge=code_challenge, code_challenge_method='S256')
pyperclip.copy(complete_auth_url) # Copy to clipboard
print(f'Complete URL w/ params - Paste In Browser: \n{complete_auth_url}')

You should now have the code...

In [ ]:
auth_code = input('Enter the code from the URL: ')

### What is the `client_assertion`?
The `client_assertion` is a JWT signed with the private key of the certificate that you created and assigned to your App Registration's authentication. 

To get the `client_assertion`, you can leverage my `jwt_sign.py` script, under `/BST/JWT_Utils/`

In [ ]:
client_assertion = input('Enter the client_assertion (JWT): ')

In [ ]:
tokens = OAuth2_Flows.request_access_token(tenant_id, client_id, redirect_uri, auth_code, client_assertion=client_assertion, code_verifier=code_verifier)
access_token = tokens[0]
refresh_token = tokens[1]
id_token = tokens[2] # id_token is not returned in regular Auth Code Flow
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}') # id_token is not returned in regular Auth Code Flow